# Charity classification analysis

Analysis using a new dataset that classifies and tags all active and inactive charities according to their activity/sector. This analysis explores how number of charities in specific activities have changed, whether specific sectors were more "trendy" at some point and whether others have died out.

For this analysis we need a number of different modules. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import os
import ast

%matplotlib inline

## Import data

We will define the root of the repository, so that we can import files from the folder more easily. 

In [2]:
root = os.path.abspath(os.path.join(os.getcwd(), ".."))

First we need to [import data](https://charityclassification.org.uk/data/data-downloads/) from the Charity Classification project. We will focus on the ICNPTSO classifications for now. 

In [3]:
active_icnptso = pd.read_csv(os.path.join(root, "data\\raw\\charities_active-icnptso.csv"))
inactive_icnptso = pd.read_csv(os.path.join(root, "data\\raw\\charities_inactive-icnptso.csv"))
icnptso = pd.read_csv(os.path.join(root, "data\\raw\\icnptso.csv"))

Then we import data from the Charity Commission register using [FindThatCharity](https://findthatcharity.uk/orgid/type/registered-charity). This includes all active and removed charities registered in the UK as of November 2021. 

In [4]:
charities = pd.read_csv(os.path.join(root, "data\\raw\\registered-charity.csv"))

C:\Users\lisa.hornung\anaconda3\envs\py3_charity\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3,7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Cleaning

### Check

First we want to get a feel for how big the different dataframes are and what variables they contain.

In [5]:
dataframes = [active_icnptso,inactive_icnptso,icnptso, charities ]

for df in dataframes:
    print(len(df))
    print(df.head(2))
    print("")
    print("")

202222
           org_id icnptso_code  icnptso_code_probability icnptso_code_source
0  GB-CHC-1000000          B32                     0.358            ml_model
1  GB-CHC-1000001          A11                     0.836            ml_model


191438
           org_id icnptso_code  icnptso_code_probability icnptso_code_source
0  GB-CHC-1000004          A11                     1.000            ml_model
1  GB-CHC-1000006          D13                     0.697            ml_model


127
  Section Group Sub-group                                             Title  \
0       A   NaN       NaN  Culture, communication and recreation activities   
1       A   A10       NaN                                  Culture and arts   

           Notes  
0  ICNPO Group 1  
1            NaN  


394035
               id                         name charityNumber companyNumber  \
0  GB-CHC-1000000  THE ROYAL ANNIVERSARY TRUST       1000000           NaN   
1  GB-CHC-1000001          THE ARTS FOUNDATION       100

### Merging

Now we are going to merge classifications for active and inactive into one dataframe.

In [6]:
all_icnptso = pd.concat([active_icnptso,inactive_icnptso])
all_icnptso.head(2)

,org_id,icnptso_code,icnptso_code_probability,icnptso_code_source
0,GB-CHC-1000000,B32,0.358,ml_model
1,GB-CHC-1000001,A11,0.836,ml_model


Then we want to merge the classifications to data from the Charity Commission register and start slowly building our dataframe (df) for analysis.

In [30]:
#select only columns we are interested in
df = charities[['id', 'name', 'charityNumber', 'dateRegistered', 'dateRemoved', 'active']]

#join classification data
df = pd.merge(df, all_icnptso, left_on= df["id"], right_on=all_icnptso["org_id"], how='left')

#remove columns we don't need
df = df.drop(columns=["key_0", "org_id"])

len(df)

394035

Now we want to clean up the icnptso categories a bit more and merge in the titles.

In [31]:
#get group
df["icnptso_group"] = df["icnptso_code"].str[0]

#get title for group
df_join = icnptso.loc[icnptso["Group"].isna()][["Section", "Title"]]
df = pd.merge(df, df_join, left_on = df["icnptso_group"], right_on=df_join["Section"], how='left')
df = df.drop(columns=["key_0", "Section"]).rename(columns={"Title": "icnptso_group_title"})

len(df)

394035

In [32]:
df.head()

,id,name,charityNumber,dateRegistered,dateRemoved,active,icnptso_code,icnptso_code_probability,icnptso_code_source,icnptso_group,icnptso_group_title
0,GB-CHC-1000000,THE ROYAL ANNIVERSARY TRUST,1000000,1990-08-03,NaN,True,B32,0.358,ml_model,B,Education services
1,GB-CHC-1000001,THE ARTS FOUNDATION,1000001,1990-08-02,NaN,True,A11,0.836,ml_model,A,"Culture, communication and recreation activities"
2,GB-CHC-1000002,THE SPENSER-MORRIS CHARITABLE FOUNDATION LIMITED,1000002,1990-08-02,NaN,True,H10,0.993,ml_model,H,Philanthropic intermediaries and voluntarism p...
3,GB-CHC-1000003,SOUTHERN AFRICA RESOURCES CENTRE,1000003,1990-08-02,NaN,True,G20,NaN,manual,G,"Civic, advocacy, political and international a..."
4,GB-CHC-1000004,THE MICHAEL VYNER TRUST,1000004,1990-08-01,2004-10-20,False,A11,1.000,ml_model,A,"Culture, communication and recreation activities"


### Dates

In [45]:
#convert columns to pandas datetime
df["dateRemoved"] = pd.to_datetime(df["dateRemoved"])
df["dateRegistered"] = pd.to_datetime(df["dateRegistered"])

#get years from date
df["yearRemoved"] = df["dateRemoved"].dt.year
df["yearRegistered"] = df["dateRegistered"].dt.year

#combine years before 1960 (as E&W database only starts then)
df["yearRemovedCombined"] = df["yearRemoved"]
df.loc[df["yearRemoved"] < 1960, "yearRemovedCombined"] = 1960 
df["yearRegisteredCombined"] = df["yearRegistered"]
df.loc[df["yearRegistered"] < 1960, "yearRegisteredCombined"] = 1960 

## Analysis

### Number of registrations by year and group

In [46]:
table = df.groupby(["yearRegisteredCombined", "icnptso_group_title"])["id"].count()
table = pd.DataFrame(table).unstack()

#export table
table.to_csv(os.path.join(root, "data\\processed\\registrations-by-year-combined-icnptso.csv"))

table

id  \
icnptso_group_title    Business, professional and labour organizations   
yearRegisteredCombined                                                   
1960                                                              11.0   
1961                                                              24.0   
1962                                                             121.0   
1963                                                              98.0   
1964                                                              62.0   
...                                                                ...   
2017                                                              18.0   
2018                                                              11.0   
2019                                                              18.0   
2020                                                              11.0   
2021                                                              12.0   

                                                                                \
icnptso_group_title    Civic, advocacy, political and international activities   
yearRegisteredCombined                                                           
1960                                                                147.0        
1961                                                                347.0        
1962                                                                989.0        
1963                                                               2236.0        
1964                                                               1190.0        
...                                                                   ...        
2017                                                                871.0        
2018                                                                601.0        
2019                                                                644.0        
2020                                                                698.0        
2021                                                                444.0        

                                                                                   \
icnptso_group_title    Community and economic development, and housing activities   
yearRegisteredCombined                                                              
1960                                                                157.0           
1961                                                                368.0           
1962                                                               1774.0           
1963                                                               2727.0           
1964                                                               1610.0           
...                                                                   ...           
2017                                                                777.0           
2018                                                                592.0           
2019                                                                689.0           
2020                                                                703.0           
2021                                                                390.0           

                                                                         \
icnptso_group_title    Culture, communication and recreation activities   
yearRegisteredCombined                                                    
1960                                                              207.0   
1961                                                              281.0   
1962                                                             1000.0   
1963                                                             1477.0   
1964                                                             1081.0   
...                                                                 ...   
2017            

### Removals per year and group

In [47]:
table = df.groupby(["yearRemovedCombined", "icnptso_group_title"])["id"].count()
table = pd.DataFrame(table).unstack()

#export table
table.to_csv(os.path.join(root, "data\\processed\\removals-by-year-combined-icnptso.csv"))

table

id  \
icnptso_group_title Business, professional and labour organizations   
yearRemovedCombined                                                   
1961.0                                                          2.0   
1962.0                                                          4.0   
1963.0                                                          NaN   
1964.0                                                          1.0   
1965.0                                                          1.0   
...                                                             ...   
2017.0                                                         16.0   
2018.0                                                         18.0   
2019.0                                                         17.0   
2020.0                                                         11.0   
2021.0                                                         12.0   

                                                                             \
icnptso_group_title Civic, advocacy, political and international activities   
yearRemovedCombined                                                           
1961.0                                                            25.0        
1962.0                                                             9.0        
1963.0                                                            14.0        
1964.0                                                            13.0        
1965.0                                                             7.0        
...                                                                ...        
2017.0                                                           467.0        
2018.0                                                           576.0        
2019.0                                                           631.0        
2020.0                                                           400.0        
2021.0                                                           363.0        

                                                                                \
icnptso_group_title Community and economic development, and housing activities   
yearRemovedCombined                                                              
1961.0                                                            24.0           
1962.0                                                             9.0           
1963.0                                                            17.0           
1964.0                                                            16.0           
1965.0                                                            16.0           
...                                                                ...           
2017.0                                                           393.0           
2018.0                                                           539.0           
2019.0                                                           573.0           
2020.0                                                           433.0           
2021.0                                                           311.0           

                                                                      \
icnptso_group_title Culture, communication and recreation activities   
yearRemovedCombined                                                    
1961.0                                                          45.0   
1962.0                                                           9.0   
1963.0                                                          11.0   
1964.0                                                          11.0   
1965.0                                                           7.0   
...                                                              ...   
2017.0                                                         555.0   
2018.0                                                         616.0   
2019.0                

### charities active by year

In [51]:
group = list(df["icnptso_group_title"].unique())

In [61]:

sum((df["yearRegistered"]<= 1960) & ( df["yearRemoved"] > 1960 | df["yearRemoved"].isna()))

672

In [ ]:
sum(df["yearRegistered"]<= 1960 )

In [ ]:
if 78 <= grade <= 89: